In [ ]:
import csv
import os
from pathlib import Path

In [ ]:
cc_files = os.listdir('data/cc/')
uc_files = os.listdir('data/uc/')

In [ ]:
with open('data/smos_oracle.txt', newline='') as oraclefile:
    oracle_reader = csv.reader(oraclefile, delimiter=',')
    
    with open('data/all_links.txt', 'w', newline='') as outfile:
        writer = csv.writer(outfile, quoting=csv.QUOTE_MINIMAL)
        
        for row in oracle_reader:
            for i,val in enumerate(row):
                row[i] = row[i].lstrip()
            for cc in cc_files:
                if cc.replace('.txt', '') in row:
                    cc_str = Path('data/cc/'+cc).read_text(encoding='latin-1')
                    for uc in uc_files:
                        uc_str = Path('data/uc/'+uc).read_text(encoding='latin-1')
                        if uc.replace('.txt', '') in row:
                            label = 1
                        else:
                            label = 0
                        writer.writerow([cc_str+uc_str, label])
                else:
                    pass        